In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from datetime import timedelta, date
from sodapy import Socrata

In [6]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
import app_tokens

In [10]:
blockface_detail = pd.read_csv('blockface_detail.csv', index_col=0)

In [11]:
blockface_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1505 entries, 0 to 1504
Data columns (total 11 columns):
blockfacename               1505 non-null object
location                    1505 non-null object
paidparkingarea             1505 non-null object
paidparkingsubarea          1147 non-null object
parkingcategory             1505 non-null object
parkingspacecount           1505 non-null int64
parkingtimelimitcategory    1504 non-null float64
sideofstreet                1505 non-null object
sourceelementkey            1505 non-null int64
latitude                    1505 non-null float64
longitude                   1505 non-null float64
dtypes: float64(3), int64(2), object(6)
memory usage: 141.1+ KB


In [12]:
blockface_detail.head()

,blockfacename,location,paidparkingarea,paidparkingsubarea,parkingcategory,parkingspacecount,parkingtimelimitcategory,sideofstreet,sourceelementkey,latitude,longitude
0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3346935...",Pioneer Square,Core,Paid Parking,4,120.0,SW,1001,47.602873,-122.334694
1,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,"{'type': 'Point', 'coordinates': [-122.3345126...",Pioneer Square,Core,Paid Parking,8,120.0,NE,1002,47.602949,-122.334513
2,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,"{'type': 'Point', 'coordinates': [-122.3351432...",Commercial Core,Waterfront,Paid Parking,7,120.0,NE,1006,47.603674,-122.335143
3,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3366575...",Commercial Core,Waterfront,Paid Parking,5,30.0,SW,1009,47.605018,-122.336658
4,1ST AVE BETWEEN MADISON ST AND SPRING ST,"{'type': 'Point', 'coordinates': [-122.3364474...",Commercial Core,Waterfront,Paid Parking,5,120.0,NE,1010,47.605101,-122.336447


In [13]:
#data is delayed 48 hrs
# socrata data keys for parking data
#2019 ytd
data_ytd = 'qktt-2bsy'
#last 30 days
data_mtd = 'rke9-rsvs'
# last 48 hours
data_48hrs = 'hiyf-7edq'

In [69]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.seattle.gov", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.seattle.gov',
                 app_tokens.getAppTokens()['seattle_gov'],
                timeout=120)





# columns that are numeric
num_cols = ['paidoccupancy',]

for ind, block_key in blockface_detail[:2]['sourceelementkey'].iteritems():
    # Number of records to read at a time
    loop_size = df_len = 1000

    # list of dataframes
    dfs = []
    i = 0
    #check to see if last record was full or not
    while loop_size == df_len:
        #print(i)
        print('ind: %d\tkey: %d\trecords read: %.3fMM' % (ind, block_key, i*loop_size/1e6), end='\r')
        # fetch results from seattle city server
        try:
            results = client.get(data_ytd,
                             sourceelementkey=block_key, 
                             select='occupancydatetime,paidoccupancy',
                             order='occupancydatetime',
                             limit=loop_size,
                             offset=loop_size * i)
        except:
            raise Error('Failed!')
        
        #convert to dataframe
        df = pd.DataFrame.from_records(results)


        # convert to appropriate data types
        df[num_cols] = df[num_cols].apply(pd.to_numeric)
        df[['occupancydatetime']] = df[['occupancydatetime']].apply(pd.to_datetime)    

        # get length of new dataframe
        df_len = len(df)
        #append dataframe to list of dataframe
        dfs.append(df)
        #update counter
        i += 1
    
    results_df = pd.concat(dfs, ignore_index=True).set_index('occupancydatetime')
    results_df = results_df.resample('15T').mean()
    results_df.dropna().to_pickle("./2019.%d.pkl" % block_key)    

In [66]:
read = pd.read_pickle('2019.1001.pkl')

In [68]:
read

,paidoccupancy
occupancydatetime,
2019-07-08 09:00:00,0.200000
2019-07-08 09:15:00,1.133333
2019-07-08 09:30:00,2.000000
2019-07-08 09:45:00,1.266667
2019-07-08 10:00:00,1.000000
2019-07-08 10:15:00,1.000000
2019-07-08 10:30:00,1.000000
2019-07-08 10:45:00,1.933333
2019-07-08 11:00:00,1.866667
